<a href="https://colab.research.google.com/github/LeoDardanello/DA_Semantic_Segmentation/blob/main/Semantic_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
import sys, os
import numpy as np
import torchvision

drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [2]:
if os.path.exists("/content/DA_Semantic_Segmentation"):
  !rm -rf "/content/DA_Semantic_Segmentation"
!git clone "https://github.com/LeoDardanello/DA_Semantic_Segmentation.git"

Cloning into 'DA_Semantic_Segmentation'...
remote: Enumerating objects: 5089, done.
remote: Counting objects: 100% (1647/1647), done.
remote: Compressing objects: 100% (1630/1630), done.
remote: Total 5089 (delta 39), reused 1614 (delta 17), pack-reused 3442
Receiving objects: 100% (5089/5089), 3.55 GiB | 43.06 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [ ]:
!7z x '/content/drive/MyDrive/AML/Cityscapes.zip'

**TRAINING**

In [3]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

# GTA 5 TEST


In [4]:
!7z x '/content/drive/MyDrive/AML/GTA5.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/AML/                                     1 file, 6215145033 bytes (5928 MiB)

Extracting archive: /content/drive/MyDrive/AML/GTA5.zip
  5% 256 Open              10% 512 Open              15% 768 Open              20% 1024 Open               25% 1280 Open               30% 1536 Open               35% 1792 Open               40% 2048 Open               46% 2304 Open               51% 2560 Open        

In [5]:
# our folder with our labels for GTA5
!unzip -qq '/content/drive/MyDrive/AMLUtils/TrainID.zip' -d '/content/GTA5/TrainID'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --num_workers 2 --dataset_train "GTA5" --dataset_test "GTA5" --checkpoint_step 1 --validation_step 10  --enable_da False --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

Training on GTA5 dataset
Generating LabelID..
100% 2500/2500 [00:00<00:00, 236928.85it/s]
Testing on GTA5 dataset
Generating LabelID..
100% 2500/2500 [00:00<00:00, 250945.55it/s]
use pretrain model /content/drive/MyDrive/STDCNet813M_73.91.tar
epoch 0, lr 0.010000:  51% 954/1874 [01:37<01:24, 10.91it/s, loss=2.376904]

In [ ]:
!rm -rf "/content/GTA5/TrainID"

# GTA TRAINING WITH CITYSCAPE TESTING

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train.py" --num_epochs 50 --batch_size 8 --learning_rate 0.001 --num_workers 2 --dataset_train "GTA5" --dataset_test "Cityscapes" --checkpoint_step 5 --validation_step 10 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_7391.tar"

# DOMAIN ADAPTATION

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"

# FOURIER DOMAIN ADAPTATION

In [ ]:
beta=0.05

In [ ]:
# get FDA images
!rm -rf "/content/GTA5/FDA"
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_{beta}/FDA.zip' -d '/content/GTA5/FDA'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --optimizer "sgd" --num_epochs 1 --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar" --lamb 0.001  --checkpoint_step 1 --enable_FDA True --beta {beta}

In [ ]:
import shutil
shutil.make_archive("/content/FDA", 'zip', "/content/GTA5/FDA")
# shutil.move("/content/FDA.zip", "/content/FDA_005")
# shutil.move("/content/source_data.txt", "/content/FDA_005")
# shutil.move("/content/source_label.txt", "/content/FDA_005")
# shutil.move("/content/FDA_005", "/content/drive/MyDrive/AMLUtils/")

## **Evaluation using all three betas**

In [ ]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.01.zip' -d '/content/FDA_run_beta0.01'

In [ ]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.05.zip' -d '/content/FDA_run_beta0.05'

In [ ]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_0.09.zip' -d '/content/FDA_run_beta0.09'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/evaluation_multibeta.py" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar"


**Generating Pseudo Label**



In [ ]:
!python3 "/content/DA_Semantic_Segmentation/evaluation_multibeta.py" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar" --use "generatepseudo"

**Finetuning**

In [ ]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/pseudolbl.zip' -d '/content/Cityscapes/Cityspaces/pseudolbl'

In [ ]:
beta=0.05

In [ ]:
!unzip -qq '/content/drive/MyDrive/AMLUtils/FDA_run_beta_{beta}.zip' -d '/content/FDA_run_beta{beta}'

In [ ]:
!python3 "/content/DA_Semantic_Segmentation/train_with_DA.py" --optimizer "sgd" --num_epochs 60 --epoch_start_i 50 --training_path "/content/FDA_run_beta0.05/latest_epoch_49_.pth" --use_pseudo_label True --save_model_path "/content/DA_Semantic_Segmentation/out" --pretrain_path "/content/drive/MyDrive/STDCNet813M_73.91.tar" --lamb 0.001  --checkpoint_step 1 --enable_FDA True --beta {beta}